# **Tarea Nº 4**

- Fecha y hora de entrega: Domingo 12 de enero, 11:59pm
- Agregue los nombres de las personas con las que discutió esta tarea: __Diego Aquino__, __Jose Tamayo__, __Franco Vela__
- Envíe su tarea haciendo el `push` de su código a su repo en GitHub Classroom: 

## **Mercado de autos usados**

Importa el conjunto de datos `neoauto_20240924.csv`
    

1. ¿Cuál es el precio promedio de los autos de cada marca (`item_brand`) publicados en cada subcategoría (`item_category_2`)?

In [ ]:
# Solución

2. ¿Cuántos autos tienen más de 100,000 kilómetros (`item_km`) pero un precio (`item_price`) menor al promedio general?

In [ ]:
# Solución

3. Encuentra el modelo (`item_name`) más caro y el más barato de cada marca (`item_brand`).

In [ ]:
# Solución

4. Encuentra las tres marcas con mayor número de autos financiados por Santander (`item_financed_by`). 

In [ ]:
# Solución

5. Agrupa las publicaciones por tipo de transmisión (`item_transmission`) y calcula la desviación estándar del precio (`item_price`) para cada tipo.  

In [ ]:
# Solución

6. ¿Cuál es el kilometraje promedio de las publicaciones con el tag "Premium" y de las publicaciones con el tag "Como nuevo"?

In [ ]:
# Solución

7. ¿Cuáles son las marcas de vehículas para los que todos sus modelos fabricados en los últimos 5 años tienen al menos 5 publicaciones cada una?

In [ ]:
# Solución

## **Evolución económica internacional**

Importa el conjunto de datos de la hoja "Data" del archivo `pwt1001.xlsx`. El diccionario de variables está disponible en la hoja "Legend" del archivo Excel.
    

1. Calcule el PBI per cápita real ajustado por poder de paridad de compra (PPP). Use la variable de PBI que se calcula por gasto y la población. ¿Cuáles son los 10 países top?

In [40]:
# Solución
import pandas as pd
data = pd.read_excel("C:/Users/HP/Documents/GitHub/tarea-4-aquino-tamayo-vela2/pwt1001.xlsx", sheet_name='Data')

data['rgdpe_pc_ppp'] = data['rgdpe'] / data['pop']

recent_years = data.groupby('countrycode')['year'].max().reset_index()
recent_data = pd.merge(recent_years, data, on=['countrycode', 'year'])

top_10_countries = recent_data.nlargest(10, 'rgdpe_pc_ppp')[['country', 'rgdpe_pc_ppp']]

print(top_10_countries)

                  country   rgdpe_pc_ppp
102            Luxembourg  112941.453342
104      China, Macao SAR  105337.890061
139                 Qatar  103445.127269
79                Ireland  102353.631443
146             Singapore   88619.305271
22                Bermuda   79967.588736
31            Switzerland   71831.637057
43         Cayman Islands   70208.345162
4    United Arab Emirates   69753.215256
26      Brunei Darussalam   67555.392597


2. Genera el ránking de los países con el mayor incremento del PBI per cápita real PPP entre los años 1990 y 2019. ¿Dónde se ubica Perú? Para los países que no tengan información para dichos años, puede utilizar datos provenientes de hasta dos años antes o después

In [7]:
# Solución
data_filtered = data[(data['year'] >= 1988) & (data['year'] <= 2021)]

data_1990 = data_filtered.loc[data_filtered['year'].sub(1990).abs().groupby(data_filtered['countrycode']).idxmin()]
data_2019 = data_filtered.loc[data_filtered['year'].sub(2019).abs().groupby(data_filtered['countrycode']).idxmin()]

merged_data = pd.merge(
    data_1990[['countrycode', 'country', 'rgdpe_pc_ppp']].rename(columns={'rgdpe_pc_ppp': 'rgdpe_pc_ppp_1990'}),
    data_2019[['countrycode', 'rgdpe_pc_ppp']].rename(columns={'rgdpe_pc_ppp': 'rgdpe_pc_ppp_2019'}),
    on='countrycode'
)

merged_data['ppp_growth'] = merged_data['rgdpe_pc_ppp_2019'] - merged_data['rgdpe_pc_ppp_1990']

ranking = merged_data.sort_values(by='ppp_growth', ascending=False).reset_index(drop=True)

peru_position = ranking[ranking['countrycode'] == 'PER'].index[0] + 1 if 'PER' in ranking['countrycode'].values else None

top_10_countries = ranking[['country', 'ppp_growth']].head(10)
print("Top 10 países con mayor incremento en PBI per cápita real PPP:")
print(top_10_countries)

if peru_position:
    print(f"\nPerú se encuentra en la posición: {peru_position}")
else:
    print("\nPerú no tiene datos disponibles para este análisis.")

Top 10 países con mayor incremento en PBI per cápita real PPP:
            country    ppp_growth
0           Ireland  82726.713568
1  China, Macao SAR  78788.824158
2             Qatar  75444.868541
3         Singapore  67092.155508
4        Luxembourg  66997.038564
5           Bermuda  48231.808944
6             Malta  35968.133995
7            Norway  34098.305410
8            Kuwait  34055.592958
9      Saudi Arabia  32993.864961

Perú se encuentra en la posición: 82


3. Para el periodo 2010-2019, calcule el promedio del índice del capital humano. ¿Qué relación existe entre esta variable y el PBI per cápita real PPP calculado previamente? ¿y el número de horas laboradas al año por los trabajadores?

In [9]:
# Solución
import seaborn as sns
import matplotlib.pyplot as plt

data['rgdpe_pc_ppp'] = data['rgdpe'] / data['pop']

data_period = data[(data['year'] >= 2010) & (data['year'] <= 2019)]

hc_avg = data_period.groupby('countrycode')['hc'].mean().reset_index()
hc_avg = hc_avg.rename(columns={'hc': 'hc_avg'})

ppp_avg = data_period.groupby('countrycode')['rgdpe_pc_ppp'].mean().reset_index().rename(columns={'rgdpe_pc_ppp': 'ppp_avg'})
avh_avg = data_period.groupby('countrycode')['avh'].mean().reset_index().rename(columns={'avh': 'avh_avg'})

combined_data = hc_avg.merge(ppp_avg, on='countrycode').merge(avh_avg, on='countrycode')

print("Resumen de datos combinados:")
print(combined_data.describe())

Resumen de datos combinados:
           hc_avg        ppp_avg      avh_avg
count  145.000000     183.000000    66.000000
mean     2.608130   21824.936268  1874.186633
std      0.688566   22370.442294   270.522674
min      1.195960     760.055772  1390.170680
25%      2.034321    5221.510286  1669.381684
50%      2.674898   14008.432573  1849.780904
75%      3.164617   30751.445052  2082.694520
max      3.738265  127462.470192  2444.905644


4. A partir de la variable del precio de los hogares, calcula los países y los años en los cuales se registró la mayor inflación anual para todo el periodo de tiempo disponible en la base de datos

In [33]:
# Solución
if {'country', 'year', 'pl_c'}.issubset(data.columns):
    data_sorted = data.sort_values(by=['country', 'year'])
    
    data_sorted['inflation'] = data_sorted.groupby('country')['pl_c'].pct_change() * 100
    
    max_inflation_row = data_sorted.loc[data_sorted['inflation'].idxmax()]
    
    print(f"Mayor inflación anual registrada:")
    print(f"País: {max_inflation_row['country']}")
    print(f"Año: {int(max_inflation_row['year'])}")
    print(f"Inflación: {max_inflation_row['inflation']:.2f}%")

Mayor inflación anual registrada:
País: Venezuela (Bolivarian Republic of)
Año: 2017
Inflación: 355.75%


C:\Users\HP\AppData\Local\Temp\ipykernel_17568\2485677633.py:7: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  data_sorted['inflation'] = data_sorted.groupby('country')['pl_c'].pct_change() * 100


5. ¿Cuáles son los países con la mayor cantidad de missing values en la variable de productividad total de factores a precios corrientes?

In [30]:
# Solución

if 'ctfp' in data.columns and 'country' in data.columns:
   
    missing_values = data.groupby('country')['ctfp'].apply(lambda x: x.isnull().sum())
    
    missing_values_sorted = missing_values.sort_values(ascending=False)
    
    print(missing_values_sorted.head(10))



country
Albania                           70
Turkmenistan                      70
Sint Maarten (Dutch part)         70
Grenada                           70
Ghana                             70
Georgia                           70
Gambia                            70
St. Vincent and the Grenadines    70
Pakistan                          70
Ethiopia                          70
Name: ctfp, dtype: int64


6.  Agrupa los datos por país y calcula el promedio de las horas trabajadas anuales (avh) y de la educación promedio (hc) para el periodo 2000-2020.

In [39]:
# Solución
filtered_data = data[(data['year'] >= 2000) & (data['year'] <= 2020)]
if {'country', 'avh', 'hc'}.issubset(filtered_data.columns):

    averages = filtered_data.groupby('country')[['avh', 'hc']].mean()

    print("Promedios de horas trabajadas anuales y educación promedio (2000-2020):")
    print(averages)

Promedios de horas trabajadas anuales y educación promedio (2000-2020):
                                            avh        hc
country                                                  
Albania                                     NaN  2.895579
Algeria                                     NaN  2.080837
Angola                                      NaN  1.399746
Anguilla                                    NaN       NaN
Antigua and Barbuda                         NaN       NaN
...                                         ...       ...
Venezuela (Bolivarian Republic of)  1874.516018  2.538335
Viet Nam                            2243.230824  2.407059
Yemen                                       NaN  1.444725
Zambia                                      NaN  2.304970
Zimbabwe                                    NaN  2.388763

[183 rows x 2 columns]


7.  Genere una variable que ordene a los países según el porcetaje de formación brutal de capital del 2019. Luego, estable diez grupos a manera de deciles. ¿En qué decil se ubica Péru? Vuelva a calcular usando la información de 1990. ¿El Perú se mantiene en el mismo décil o cambia?

In [41]:
# Solución
data_2019 = data[data['year'] == 2019]
data_1990 = data[data['year'] == 1990]

if {'country', 'csh_i'}.issubset(data.columns):
    data_2019 = data_2019.sort_values(by='csh_i', ascending=False)
    data_2019['decile_2019'] = pd.qcut(data_2019['csh_i'], 10, labels=False) + 1  

    data_1990 = data_1990.sort_values(by='csh_i', ascending=False)
    data_1990['decile_1990'] = pd.qcut(data_1990['csh_i'], 10, labels=False) + 1  
    
    peru_2019 = data_2019[data_2019['country'] == 'Peru']['decile_2019'].values
    peru_1990 = data_1990[data_1990['country'] == 'Peru']['decile_1990'].values

   
    print(f"Perú en 2019 está en el decil: {peru_2019[0] if len(peru_2019) > 0 else 'No disponible'}")
    print(f"Perú en 1990 está en el decil: {peru_1990[0] if len(peru_1990) > 0 else 'No disponible'}")

    if peru_2019 and peru_1990:
        if peru_2019[0] == peru_1990[0]:
            print("Perú se mantiene en el mismo decil en 1990 y 2019.")
        else:
            print("Perú cambia de decil entre 1990 y 2019.")
    else:
        print("No hay información suficiente para Perú en uno o ambos años.")

Perú en 2019 está en el decil: 5
Perú en 1990 está en el decil: 4.0
Perú cambia de decil entre 1990 y 2019.


## **Transacciones Financieras**

Importa el conjunto de datos `base_financiera.csv`
    

8. Filtra todas las transacciones de tipo "Debit" en las que el monto sea mayor a 200, y calcula el monto total de esas transacciones. Además, determina cuántos clientes únicos realizaron esas transacciones.

In [ ]:
# Solución

9. Identifica las transacciones con valores nulos en la columna (`Amount`) . Rellena esos valores con la mediana de los montos por tipo de transacción (`Transaction_Type`) y luego calcula el monto total de transacciones por región (`region`) .

In [ ]:
# Solución

10. Busca transacciones duplicadas considerando las columnas (`Customer_ID`) y (`Date`). Elimina los duplicados manteniendo solo la primera ocurrencia. Luego, agrupa las transacciones restantes por (`Customer_ID`) y calcula el monto promedio y la cantidad de transacciones por cliente.

In [ ]:
# Solución